# Dynamic Limit Orders

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import yfinance as yf
from ib_insync import *
import nest_asyncio
import logging
import datetime
import schedule
import time
import threading

In [ ]:
nest_asyncio.apply()

# Connect to TWS (or gateway)
ib = IB()

ib.connect('127.0.0.1', 7497, clientId=1111) #7497 - Paper Trading, 7496 - Live trading

In [18]:
#Define ES contract
contract = Future('ES', '202506', 'CME')

# Or for stocks:
#contract = Stock('TSLA', 'SMART', 'USD')

# Qualify the contract (ensures IB recognizes it)
ib.qualifyContracts(contract)

[Future(conId=620731015, symbol='ES', lastTradeDateOrContractMonth='20250620', multiplier='50', exchange='CME', currency='USD', localSymbol='ESM5', tradingClass='ES')]

In [20]:
quote = ib.reqMktData(contract, "", False, False)

In [ ]:
# Global variable to track position
position_size = 0

# Define a function to handle position updates
def update_position(account, contract, pos, avgCost):
    global position_size
    # Check if this is the contract we're trading (adjust comparison as needed)
    if contract.symbol == "ES":  # Replace with your contract's symbol
        position_size = pos
        print(f"Position updated: {position_size}")

# Assign the callback to the wrapper
ib.wrapper.position = update_position

my_bid = quote.bid - 1.00
order = LimitOrder("BUY", 1, my_bid)
ib.placeOrder(contract, order)

# Request position updates
ib.reqPositions()

# Start the IB event loop in a separate thread
threading.Thread(target=ib.run, daemon=True).start()

# Loop every 1 sec
while True:
    ib.sleep(0.5)
    
    if position_size == 0:  # Only proceed if no position
        # If best bid > my bid, cancel & update my bid
        if quote.bid > my_bid:
            new_bid = quote.bid - 0.25
            new_order = LimitOrder("BUY", 1, new_bid)
         
            # Cancel the old order and place the new order
            ib.cancelOrder(order)
            ib.placeOrder(contract, new_order)
            print("Cancelled order/placed new order:", new_bid)
            
            # Update the current bid and order for subsequent iterations
            my_bid = new_bid
            order = new_order
        else:
            print("Quote.bid not > my_bid.")
    else:
        print("Position already held, skipping order placement.")